In [61]:
import pickle
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

pd.options.display.max_rows = 100

In [70]:
# shortest path (physical time)
from networkx.algorithms.shortest_paths.generic import shortest_path_length

with open('../Subway_base_network/graph_stNo_as_node.pkl', 'rb') as f:
    G_stNo = pickle.load(f)

physical_times = dict(shortest_path_length(G_stNo, weight='weight')) 

In [87]:
# name_to_no

df = pd.read_csv("../Subway_base_network/seoul.txt", delimiter=" ", names=["no", "name", "line"], encoding="UTF-8",)
## only nodes
df = df.iloc[:705,]

def name_to_no(name, line):
    x = df[df["name"] == name]
    y = x[x['line'] == line]
    return y['no'].values[0]

name_to_no("사당", '2')

'226'

In [94]:
df['line'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', 'S1', 'I1', 'I2',
       'K1', 'K2', 'K2_', 'K4', 'P1', 'S', 'AREX', 'D', 'U1', 'Y1'],
      dtype=object)

In [104]:
# load data
data = pd.read_csv("data.csv")

# convert line names and station names
line_convert_table = {"1호선": "1", "2호선": "2", "3호선": "3", "4호선": "4", "5호선": "5", "6호선": "6", "7호선": "7", "8호선": "8", "9호선": "9", "중앙선": "K1", '우이신설선': 'S1', '분당선': "K2", '수인선': 'K2_'}
name_convert_table = {'가천대': '가천대역', '신천': '잠실새내', '이수': '총신대입구(이수)','청량리(서울시립대입구)':'청량리','강변(동서울터미널)':'강변'}

data.replace(line_convert_table, inplace=True)
data.replace(name_convert_table, inplace=True)

# 2개의 다른 신촌역 분리
data.loc[data['지하철역']=='신촌'].iloc[0]['지하철역'] = '신촌' # 2호선 신촌
data.loc[data['지하철역']=='신촌'].iloc[1]['지하철역'] = '신촌(경의선)'

In [118]:
data = pd.read_csv("data.csv")

data.loc[data['지하철역'] == '양평(중앙선)']

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
578,201803,중앙선,양평(중앙선),20,2,1529,131,2974,2209,13161,...,4861,157,1912,0,0,0,0,0,0,20180403


In [106]:
no_list = []
for i in range(data.shape[0]):
    print(i)
    row = data.iloc[i]
    no_list.append(name_to_no(row['지하철역'],row['호선명']))

0
1
2
3
4
5
6
7
8
9
10
11
12
13


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# add daily sum
data['sum'] = data.sum(axis=1)

In [73]:
# sum duplicated rows(환승역)
data_org = data.copy()
data = data.groupby('지하철역').sum()

# drop columns which we don't need
data.drop(columns=['사용월','작업일자'], inplace=True)

# sort by sum
data.sort_values('sum', ascending=False, inplace=True)

In [81]:
# V represents the ratio of "passsenger flow at the station : mean passenger flow".
data['V'] = data['sum'] / data_sum.mean(axis=0)['sum']

In [82]:
# Potential
for row in data:
    

,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,07시-08시 하차인원,08시-09시 승차인원,08시-09시 하차인원,...,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,sum,V
지하철역,,,,,,,,,,,,,,,,,,,,,
서울역,1318,21,22986,10577,29179,83489,76053,182308,132154,400485,...,6791,18168,2,1063,0,0,0,0,127851724,5.172036
공덕,218,1,9839,3439,19999,20999,58112,78487,79065,224580,...,3727,10803,2,378,1,0,0,0,83865121,3.392629
디지털미디어시티,182,4,10184,3529,23694,19640,72913,58263,77713,179169,...,1806,9114,2,524,0,0,0,0,83277088,3.368841
김포공항,53,1,5806,7154,12086,35336,33388,54358,44622,58349,...,798,4428,4,316,0,0,1,0,83174765,3.364702
홍대입구,2086,34,47598,5569,48204,30499,92455,56543,115693,156566,...,19736,29399,7,2499,1,1,0,0,67265739,2.721128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
신원,0,0,94,8,237,59,623,279,493,468,...,4,58,0,0,0,0,0,0,20396144,0.825093
굴봉산,0,0,104,5,100,21,277,68,241,267,...,1,18,0,0,0,0,0,0,20392004,0.824926
백양리,1,0,101,17,37,44,66,142,130,411,...,1,28,0,0,0,0,0,0,20390489,0.824865
